In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("../data/processed/clean_listings.gz")
df = df.astype({'id': 'object', 'host_id': 'object', 'latitude': 'float', 'longitude': 'float', 'accommodates': 'float', 'bedrooms': 'float', 'beds': 'float', 'price': 'float', 
               'minimum_nights': 'int', 'maximum_nights': 'int', 'number_of_reviews': 'int', 'review_scores_rating': 'float',})
# print(len(df), len(df.columns))
# df.head()
df.dtypes

id                       object
name                     object
host_id                  object
host_since               object
host_response_time       object
host_response_rate       object
host_acceptance_rate     object
latitude                float64
longitude               float64
property_type            object
room_type                object
accommodates            float64
bedrooms                float64
beds                    float64
price                   float64
minimum_nights            int64
maximum_nights            int64
number_of_reviews         int64
review_scores_rating    float64
city                     object
state                    object
dtype: object

In [4]:
# state_data = df[["state", "id", "price", "number_of_reviews", "review_scores_rating", "city"]]
# state_data['date'] = pd.to_datetime(state_data['date'])
# state_data['month'] = state_data['date'].dt.month
# state_data['year'] = state_data['date'].dt.year
# state_data = state_data.drop('date', axis=1)

states = df[["state", "id", "price", "number_of_reviews", "review_scores_rating", "city"]].groupby(['state']).agg(
                        number_listings=('id', 'nunique'),
                        review_counts=('number_of_reviews', 'sum'),
                        median_price=('price', 'median'),
                        average_rating=('review_scores_rating', 'mean'),
                        cities=('city', 'nunique')).reset_index()

In [5]:
# convert state full name to code
state_dict = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}
states['code'] = states['state'].map(state_dict)
states['Reviews/Listing'] = states['review_counts']/states['number_listings']
states['Listing/City'] = states['number_listings']/states['cities']

states.to_csv('../data/analytical/state_listings.gz', compression='gzip', index=False)

In [6]:
import plotly.graph_objects as go

state = states.copy()

for col in state.columns:
    state[col] = state[col].astype(str)

state['text'] = state['state'] + '<br>' + \
    'Number of cities ' + state['cities'] + '<br>' + \
    'Total reviews ' + state['review_counts'] + '<br>' + \
    'Reviews per Listing ' + state['Reviews/Listing'] + '<br>' + \
    'Listings per City ' + state['Listing/City']

fig = go.Figure()

fig.add_trace(
    go.Choropleth(
    locations=state['code'],
    z=state['median_price'].astype(float),
    locationmode='USA-states',
    colorscale='Oryel',
    autocolorscale=False,
    text=state['text'], # hover text
    marker_line_color='white', # line markers between states
    colorbar_title="Median USD",
    visible=True,
))

fig.add_trace(
    go.Choropleth(
    locations=state['code'],
    z=state['number_listings'].astype(int),
    locationmode='USA-states',
    colorscale='Oranges',
    autocolorscale=False,
    text=state['text'], # hover text
    marker_line_color='white', # line markers between states
    colorbar_title="Listing Counts",
    visible=False,
))

fig.add_trace(
    go.Choropleth(
    locations=state['code'],
    z=state['average_rating'].astype(float),
    locationmode='USA-states',
    colorscale='peach',
    autocolorscale=False,
    text=state['text'], # hover text
    marker_line_color='white', # line markers between states
    colorbar_title="Mean Rating",
    visible=False,
))

button_height = 0.2
x1_loc = 0.00
y1_loc = 1

fig.update_layout(
    title_text='2021/6-2022/5 US States Airbnb Data<br>(Hover for other information)',
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
    updatemenus=[
        # BUTTON-1: OPACITY
        dict(
            # NOTICE THAT THE OPTION ARE A LIST OF DICTIONARIES
            # IMPORTANT: args specifies the key-value pairing for what to change
            buttons=[
                dict(
                    label="Median Price",               # LABEL SHOWN TO USER
                    method="update",           # MODIFICATION TYPE (SEE ABOVE)
                     args=[{"visible": [True, False, False]},
                           {"subtitle": "Median Price (states)"}]
                     ),
                dict(
                    label="Number of listings",            # LABEL SHOWN TO USER
                    method="update",                    # MODIFICATION TYPE (SEE ABOVE)
                     args=[{"visible": [False, True, False]},  # BOOLEAN VALUES FOR EACH TRACE
                           {"subtitle": "Number of listings (states)"}]
                     ),
                dict(
                label="Average Rating",               # LABEL SHOWN TO USER
                method="update",           # MODIFICATION TYPE (SEE ABOVE)
                    args=[{"visible": [False, False, True]},
                        {"subtitle": "Average Rating (states)"}]
                    ),              
            ],
            # PLACEMENT AND META DATA FOR THE BUTTON
            direction="down",
            showactive=True,  # HIGHLIGHTS ACTIVE DROPDOWN ITEM OR ACTIVE BUTTON IF TRUE
            pad={"r": 10, "t": 10},  # PADDING
            x=x1_loc,  # POSITION
            y=y1_loc,
            xanchor="left",  # ANCHOR POINT
            yanchor="top",
        )
    ]
)

fig.show()

California has the most listings overall from Airbnb dataset, but they are spread across eight cities which could have significantly different characteristics and patterns in terms of ratings, prices, and property types. New York City and Hawaii, on the other hand, are both famous tourist destinations, and they are expected to have high listing volumes and higher median prices. 

While Hawaii does have the highest median price among all states, and Rhode Island has the second-highest median price, we cannot overlook the importance of ratings in generating revenue.

Ratings provide a comprehensive indicator of a traveler's overall experience with an Airbnb listing, taking into account many different aspects such as location, amenities, cleanliness, and host communication. As such, ratings are assumed to be highly correlated with revenue. Therefore,  we have chosen the average rating as a key factor to narrow down our analysis in order to identify potential strategies and provide valuable insights to households to increase revenue. Although Montana has the highest average rating, the number of listing is too small. We will focus on analyzing North Carolina which has the second high average rating with a relatively large number of listings.

In [2]:
nc=pd.read_csv("../data/processed/clean_airbnb.gz")
nc = nc.astype({'price': 'float', 'latitude': 'float', 'latitude': 'float', 'latitude': 'float', 'longitude': 'float', 
              'accommodates': 'int', 'bedrooms': 'float', 'beds': 'float', 'minimum_nights': 'int', 'maximum_nights': 'int', 
              'number_of_reviews': 'int', 'calculated_host_listings_count': 'int', 'calculated_host_listings_count_entire_homes': 'int', 
              'calculated_host_listings_count_private_rooms': 'int', 'calculated_host_listings_count_shared_rooms': 'int', 'reviews_per_month': 'float'})
nc['date'] = pd.to_datetime(nc['date'])
print(len(nc), len(nc.columns))
nc.head()

1001925 28


,id,name,price,date,available,room_type,accommodates,bedrooms,beds,amenities,...,host_acceptance_rate,minimum_nights,maximum_nights,number_of_reviews,review_scores_rating,reviews_per_month,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms
0,108061,Walk to stores/parks/downtown. Fenced yard/Pet...,100.0,2023-03-19,f,Entire home/apt,2,1.0,1.0,"[""Coffee maker"", ""Cleaning products"", ""Extra p...",...,71%,30,365,92,4.51,0.66,2,2,0,0
1,108061,Walk to stores/parks/downtown. Fenced yard/Pet...,100.0,2023-03-20,f,Entire home/apt,2,1.0,1.0,"[""Coffee maker"", ""Cleaning products"", ""Extra p...",...,71%,30,365,92,4.51,0.66,2,2,0,0
2,108061,Walk to stores/parks/downtown. Fenced yard/Pet...,100.0,2023-03-21,f,Entire home/apt,2,1.0,1.0,"[""Coffee maker"", ""Cleaning products"", ""Extra p...",...,71%,30,365,92,4.51,0.66,2,2,0,0
3,108061,Walk to stores/parks/downtown. Fenced yard/Pet...,100.0,2023-03-22,f,Entire home/apt,2,1.0,1.0,"[""Coffee maker"", ""Cleaning products"", ""Extra p...",...,71%,30,365,92,4.51,0.66,2,2,0,0
4,108061,Walk to stores/parks/downtown. Fenced yard/Pet...,100.0,2023-03-23,f,Entire home/apt,2,1.0,1.0,"[""Coffee maker"", ""Cleaning products"", ""Extra p...",...,71%,30,365,92,4.51,0.66,2,2,0,0


In [4]:
# nc['month'] = nc['date'].nc.month
# nc['year'] = nc['date'].dt.year
# nc['month-year'] = nc['dates'].nc.to_period('m')

# change available to 1 or 0 for further calculation
nc['fill'] = np.where(nc['available']=='f', 1, 0)

# calculate the revenue
nc['revenue'] = nc['price']*nc['fill']

In [34]:
nc.columns

Index(['id', 'name', 'price', 'date', 'available', 'room_type', 'accommodates',
       'bedrooms', 'beds', 'amenities', 'latitude', 'longitude',
       'property_type', 'host_id', 'host_since', 'host_location',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'minimum_nights', 'maximum_nights', 'number_of_reviews',
       'review_scores_rating', 'reviews_per_month',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'fill', 'revenue'],
      dtype='object')

In [48]:
dates = nc[['id', 'price', 'number_of_reviews', 'date','room_type','fill','revenue']].groupby(['date','room_type','fill']).agg(
    listings=('id', 'nunique'),
    total_revenue=('revenue', 'sum'),
    average_price=('price', 'mean'),
    total_reviews=('number_of_reviews', 'sum'),).reset_index()
    # occupancy=('fill','sum')
dates

,date,room_type,fill,listings,total_revenue,average_price,total_reviews
0,2023-03-19,Entire home/apt,0,842,0.0,195.204276,83206
1,2023-03-19,Entire home/apt,1,1561,273622.0,175.286355,156654
2,2023-03-19,Hotel room,0,3,0.0,372.666667,27
3,2023-03-19,Hotel room,1,9,4235.0,470.555556,352
4,2023-03-19,Private room,0,98,0.0,125.438776,8906
...,...,...,...,...,...,...,...
2711,2024-03-17,Hotel room,0,6,0.0,294.333333,275
2712,2024-03-17,Hotel room,1,6,3587.0,597.833333,104
2713,2024-03-17,Private room,0,111,0.0,133.027027,10624
2714,2024-03-17,Private room,1,213,22699.0,106.568075,24554


In [41]:
dates.to_csv('../data/analytical/date_fill_roomType.gz', compression='gzip', index=False)

,date,room_type,number_listings,total_revenue,median_price,review_counts,occupancy
0,2023-03-19,Entire home/apt,2403,273622.0,141.0,239860,1561
1,2023-03-19,Hotel room,12,4235.0,345.0,379,9
2,2023-03-19,Private room,324,25172.0,79.0,35178,226
3,2023-03-19,Shared room,6,250.0,33.5,1030,1
4,2023-03-20,Entire home/apt,2403,214226.0,141.0,239860,1252
...,...,...,...,...,...,...,...
1455,2024-03-16,Shared room,6,418.0,33.5,1030,6
1456,2024-03-17,Entire home/apt,2403,198196.0,141.0,239860,1221
1457,2024-03-17,Hotel room,12,3587.0,345.0,379,6
1458,2024-03-17,Private room,324,22699.0,79.0,35178,213


In [87]:
# Create figure
fig = go.Figure()

type_room = dates['room_type'].unique()
ls = []
for i in range(5):
    if i < len(type_room):
        df_comb = dates.loc[(dates['fill']==1) & (dates['room_type']==type_room[i]), ['date','total_reviews','listings','total_revenue','room_type']].groupby(
            ['date',]).agg(listings=('listings', 'count'),
                            revenue=('total_revenue', 'sum'),
                            reviews=('total_reviews', 'sum')).reset_index()
        ls.append(type_room[i])
    else: 
        df_comb = dates.loc[(dates['fill']==1), ['date','total_reviews','listings','total_revenue']].groupby(
            ['date',]).agg(listings=('listings', 'count'),
                            revenue=('total_revenue', 'sum'),
                            reviews=('total_reviews', 'sum')).reset_index()
        ls.append('All types')
    fig.add_trace(
        go.Scatter(
            uid=f"{i}",
            x=list(df_comb.date), 
            y=list(df_comb.revenue),
            name=f"{ls[i]}",
            visible=False,
            # colorbar_title=f"Revenue of {type_room[i]}",
            )
            )

fig.data[4].visible = True

# ADD DROPDOWN TO CHANGE TYPE
button_height = 0.06
x1_loc = 0.16
y1_loc = 1.25
buttons = []
for i in range(len(ls)):
    button = dict(
        label=ls[(len(ls)-1-i)],
        method="update",
        args=[{"visible": [False] * len(fig.data)}]
    )
    button["args"][0]["visible"][i]=True
    buttons.append(button)

# Add range slider
fig.update_layout(
    # Set title
    title_text="Total Revenue for different room type",
    updatemenus=[
        dict(
            # IMPORTANT: args specifies the key-value pairing for what to change
            buttons=buttons,
            # PLACEMENT AND META DATA FOR THE BUTTON
            direction="right",
            showactive=True,  # HIGHLIGHTS ACTIVE DROPDOWN ITEM OR ACTIVE BUTTON IF TRUE
            pad={"r": 10, "t": 10},  # PADDING
            x=x1_loc,  # POSITION
            y=y1_loc,
            xanchor="left",  # ANCHOR POINT
            yanchor="top",
        )
    ],
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=3,
                     label="3m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

# fig.update_layout(template="plotly_white")

fig.show()

The plots for Airbnb reservations Revenue in North Carolina shows some interesting patterns. Firstly, revenue tends to increase significantly from Friday to Sunday and is at its lowest on Tuesdays and Wednesdays. Additionally, August and September have relatively low reservation rates compared to other months.

Most of the Airbnb listings in North Carolina are private rooms and entire home/apartment types, with the majority of revenue coming from entire home/apartment rentals. It's not surprising that Hotel Room and Shared room types have lower revenues. The booking patterns for private rooms and entire home/apts differ based on the month. For private rooms, revenue tends to decrease from April to May, increase from mid-June, and further increase from mid-September. On the other hand, for entire home/apts, bookings decrease until mid-September.

To increase revenue, households can adjust their pricing strategies by increasing prices during weekends, and decreasing them during weekdays to attract more travelers and increase occupancy rates, matching the booking patterns. Private room households may consider raising their prices after mid-July when booking rates tend to increase. For entire home/apt listings, households may consider converting them to rent private rooms inside the apartment/house instead of renting the entire space, as demand for entire home/apt bookings decreases until mid-September.

In [5]:
import altair as alt
alt.data_transformers.enable('default', max_rows=None)

DataTransformerRegistry.enable('default')

In [6]:
nc_listings = nc[['id', 'revenue', 'price', 'room_type', 'accommodates', 'bedrooms', 'beds','number_of_reviews','review_scores_rating','fill']].groupby(
                 ['id', 'price', 'room_type', 'accommodates', 'bedrooms', 'beds','number_of_reviews','review_scores_rating']).agg(
                        revenue=('revenue', 'sum'),
                        fill=('fill', 'sum'),
                        # median_price=('price', 'nunique'),
                        # average_rating=('review_scores_rating', 'mean'),
                        ).reset_index()
# The occupancy rate was capped at 70% - a relatively high, but reasonable number for a highly occupied "hotel".
# http://insideairbnb.com/data-assumptions/
# Not consider those listings hve higher than 70% occupancy rate
occupancy_rate=0.7
nc_listings=nc_listings[nc_listings['fill'] < occupancy_rate*365]
# Remove outliers
nc_listings=nc_listings[nc_listings['revenue'] < 1000000]

nc_listings

,id,price,room_type,accommodates,bedrooms,beds,number_of_reviews,review_scores_rating,revenue,fill
0,108061,100.0,Entire home/apt,2,1.0,1.0,92,4.51,11900.0,119
1,155305,100.0,Entire home/apt,2,1.0,1.0,383,4.59,16800.0,168
4,209068,100.0,Entire home/apt,4,1.0,2.0,60,4.87,13400.0,134
7,304379,289.0,Entire home/apt,4,2.0,2.0,24,4.90,35258.0,122
8,353092,69.0,Entire home/apt,4,1.0,3.0,529,4.85,15456.0,224
...,...,...,...,...,...,...,...,...,...,...
2737,828558764470227964,139.0,Entire home/apt,2,1.0,1.0,3,5.00,139.0,1
2738,829737607287146165,122.0,Entire home/apt,4,2.0,2.0,2,5.00,244.0,2
2739,831250943446647268,78.0,Entire home/apt,2,1.0,1.0,2,5.00,2808.0,36
2740,837831502603860900,192.0,Entire home/apt,6,3.0,3.0,1,5.00,2688.0,14


In [8]:
nc_listings.to_csv('../data/analytical/nc_listings.gz', compression='gzip', index=False)

In [27]:
selection = alt.selection_single(fields=['room_type'],name='Random')
color = alt.condition(selection,
                      alt.value('lightblue'),
                      alt.value('lightgray'))

bar=(alt.Chart(nc_listings)
 .mark_bar()
 .encode(x='sum(revenue):Q',
         y=alt.X('room_type:N',
         sort=alt.EncodingSortField(field='revenue', op='sum', 
                            order='descending')),
         color=color
        )
).add_selection(selection).properties(
        width=800,
        height=100,
    ).interactive()

bar.title ="Total Revenue for different room type"
bar.encoding.x.title = 'Room Type'
bar.encoding.y.title = 'Revenue'


color1 = alt.condition(selection,
                      alt.Color('room_type:N'),
#                       alt.value('steelblue'),
                      alt.value('lightgray'))
opacity = alt.condition(selection, alt.value(0.5), alt.value(0.05))

scatter_plot1 = alt.Chart(nc_listings).mark_circle(clip=True).encode(
    alt.X('number_of_reviews:Q', scale=alt.Scale(zero=False, domain=(-1, 1100))), #, type='quantitative' .scale(domain=(0.1, 5.5))
    alt.Y('revenue', scale=alt.Scale(zero=False, padding=1),),
    color=color1,
    opacity=opacity,
    size=alt.Size('price:Q'),
#     opacity=alt.value(0.5),
    tooltip=['revenue:Q', 'number_of_reviews:Q', 'room_type:N', 'beds:O'],
    ).properties(
        width=300,
        height=300,
    ).interactive()

scatter_plot2 = alt.Chart(nc_listings).mark_point().encode(
    alt.X('review_scores_rating:Q', scale=alt.Scale(zero=False,domain=(2.5, 5.1))), #, type='quantitative'
    alt.Y('revenue', scale=alt.Scale(zero=False, padding=1),),
    color=color1,
    opacity=opacity,
    size='price',
#     opacity=alt.value(0.5),
    tooltip=['revenue:Q', 'review_scores_rating:Q', 'room_type:N', 'beds:O'],
    ).properties(
        width=300,
        height=300,
    ).interactive()

# Configure the ticks
tick_axis = alt.Axis(labels=False, domain=False, ticks=False)

x_ticks1 = alt.Chart(nc_listings).mark_tick().encode(
    alt.X('number_of_reviews:Q', title='', axis=tick_axis, scale=alt.Scale(zero=False, domain=(-1, 1100))),
    alt.Y('room_type:N', title='', axis=tick_axis),
    color=color1
).properties(
        width=300,
        height=70,
).interactive()

x_ticks2 = alt.Chart(nc_listings).mark_tick().encode(
    alt.X('review_scores_rating:Q', title='', scale=alt.Scale(zero=False,domain=(2.5, 5.1)), axis=tick_axis),
    alt.Y('room_type:N', title='', axis=tick_axis),
    color=color1
).properties(
        width=300,
        height=70,
).interactive()

y_ticks = alt.Chart(nc_listings).mark_tick().encode(
    alt.X('room_type:N', title='', axis=tick_axis),
    alt.Y('revenue', title='', axis=tick_axis),
    color=color1
).properties(
        width=70,
        height=300,
).interactive()

# Build the chart
bar & (y_ticks | ((scatter_plot1 & x_ticks1) | (scatter_plot2 & x_ticks2)) )

alt.VConcatChart(...)

The scatterplot charts with number of reviews and review scores as x-axes and revenue as y-axis, with price as point size, offer some valuable insights for North Carolina Airbnb hosts. One of the key findings is that some listings with higher prices and fewer reviews generate more revenue, likely due to having more beds available. However, for most listings with 0-100 reviews, increasing the number of reviews does not necessarily lead to higher revenue, especially for those with fewer beds.

The scatterplot chart also reveals that most listings in North Carolina have review scores higher than 4.5, and higher review scores tend to generate higher revenue. This indicates that hosts should prioritize providing top-notch services to guests to receive positive reviews, which can ultimately boost their revenue. By maintaining high-quality services and keeping guests happy, hosts can improve their review scores and increase their revenue in the long run.

In [24]:
multi_boxplots = alt.Chart(nc_listings).mark_boxplot(size=15, extent=1).encode(
    alt.X(alt.repeat('column'), type='ordinal'),
    alt.Y('revenue:Q'),
    # color='room_type:N',
    opacity=alt.value(0.8),
    tooltip=['revenue:Q', 'number_of_reviews:Q', 'review_scores_rating:Q','room_type:N'],
    ).properties(
        width=300,
        height=300
    ).repeat(
        column=['accommodates', 'bedrooms', 'beds']
    ).interactive()

multi_boxplots